## This program creates Neighborhood Delination across different time period


In [4]:
#!/usr/bin/env python
# coding: utf-8
import json, math, copy, sys
from geosnap.io import store_ltdb
from geosnap import Community, datasets
from geosnap.io import store_census
import pandas as pd
import shapely.wkt
import shapely.geometry
from datetime import datetime
from datetime import timedelta
from pathlib import Path
import urllib.parse
import webbrowser
import os
import pprint
from sklearn.preprocessing import minmax_scale
import numpy as np
from scipy import stats
from notebook import notebookapp
from IPython.core.display import display, HTML
import geopandas as gpd

Loading manifest: 100%|███████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2002.05entries/s]


### Variable names in "variable" column are needed to use built-in data from geosnap.

In [5]:
datasets.codebook().head()

,variable,label,formula,ltdb,ncdb,census_1990_form,census_1990_table_column,census_2000_form,census_2000_table_column,acs,category,notes
0,geoid,FIPS code,NaN,geoid,GEO2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,n_mexican_pop,persons of Mexican parentage or ancestry,NaN,mex,MEXIC,SF1,P0090001,SF1,PCT011004,B03001_004E,Ethnicity & Immigration,NaN
2,n_cuban_pop,persons of Cuban parentage or ancestry,NaN,cuban,CUBAN,SF1,P0090004,SF1,PCT011006,B03001_006E,Ethnicity & Immigration,NaN
3,n_puerto_rican_pop,persons of Puerto Rican parentage or ancestry,NaN,pr,PRICAN,SF1,P0090003,SF1,PCT011005,B03001_005E,Ethnicity & Immigration,NaN
4,n_russian_pop,persons of Russian/USSR parentage or ancestry,NaN,ruanc,NaN,SF3,P0330022,SF3,PCT016064+PCT016053+PCT016052+PCT016037,B04004_064E,Ethnicity & Immigration,ruancXX (page 17 of LTDB codebook) suggests th...


### Run clustering using Data received from Users. csv and shapefile received from the user in this example

In [69]:
param = {
    'title': "Longitudinal Neighborhood Change, Cook County (tract level)",
    'subject': "NEIGHBORHOOD",
    'filename_suffix': "Cook_Temporal_1980_1990_2000_2010_from_file",           
    'inputCSV': "attributes/LTDB_2018_1990_2000_2010__tract_Cook_byTract_normalized.csv",   
    'shapefile': "shp/Cook_County_Tract.shp", 
    'layers': [1980, 1990, 2000, 2010],           # Available years: 1970, 1980, 1990, 2000 and 2010
    'method': "kmeans",                          # Aspatial Clustering: affinity_propagation, gaussian_mixture, hdbscan, kmeans, spectral, ward
                                                 # Spatial Clustering: azp, max_p, skater, spenc, ward_spatial   
    'nClusters': 6,                              # This option should be commented out for affinity_propagation, hdbscan and max_p 
    'variables': [      #Variables are column headers of your CSV
            "p_nonhisp_white_persons",
            "p_nonhisp_black_persons",
            "p_hispanic_persons",
            "p_asian_persons",
            "p_foreign_born_pop",
            "p_edu_college_greater",
            "p_unemployment_rate",
            "p_employed_manufacturing",
            "p_poverty_rate",
            "p_vacant_housing_units",
            "p_owner_occupied_units",
            "p_housing_units_multiunit_structures",
            "median_home_value",
            "p_structures_30_old",
            "p_household_recent_move",
            "p_persons_under_18",
            "p_persons_over_60",       
                 ],
    #'Sequence': {'seq_clusters': 5, 'dist_type': 'tran'},
    'Maps_of_Categorical_Data': True,    #Choropleth Maps visualizing categorical data
    'Stacked_Chart': True,               #Visualize Temporal Change over periods
    'Parallel_Categories_Diagram': True, #Quantifies Changes over multiple periods
    'Chord_Diagram': False               #Quantifies Change between two periods
}

### Run clustering using Data received from Users. csv and shapefile received from the user in this example

In [72]:
param = {
    'title': "Neighborhood, Cook County (tract level)",
    'subject': "NEIGHBORHOOD",
    'filename_suffix': "Cook_2018_from_ACS", 
    'inputCSV': "attributes/ACS_2018_5year__tract_Cook_byTract_normalized.csv",   
    'shapefile': "shp/Cook_County_Tract.shp", 
    'layers': [2018],           
    'method': "kmeans",                          # Aspatial Clustering: affinity_propagation, gaussian_mixture, hdbscan, kmeans, spectral, ward
                                                 # Spatial Clustering: azp, max_p, skater, spenc, ward_spatial   
    'nClusters': 6,                              # This option should be commented out for affinity_propagation, hdbscan and max_p 
    'variables': [     #Variables are column headers of your CSV
            "Median monthly housing costs",
            "% below poverty",
            "% unemployed",
            "% with 4year college degree",
            "% manufacturing",
            "% service industry",
            "% structures more than 30 years old",
            "% households moved <10 years ago",
            "% multiunit structures",
            "% owner occupied housing",
            "% vacant housing",
            "% > 60 years old",
            "% < 18 years old",
            "% white",
            "% Asian",
            "% Hispanic",
            "% black",
            "% foreign born"       
                 ],
    'Maps_of_Categorical_Data': True,     #Choropleth Maps visualizing categorical data
    'Stacked_Chart': False,               #Visualize Temporal Change over periods
    'Parallel_Categories_Diagram': False, #Quantifies Changes over multiple periods
    'Chord_Diagram': False                #Quantifies Change between two periods
}

### Run clustering using Data received from Users. csv and shapefile received from the user in this example

In [125]:
param = {
    'title': "Neighborhood, Cook County (zipcode level)",
    'subject': "NEIGHBORHOOD",
    'filename_suffix': "Cook_2018_from_ACS_zipcode", 
    'inputCSV': "attributes/ACS_2018_5year__zipcode_Cook_byZipcode_normalized.csv",   
    'shapefile': "shp/zipcode_Cook_County.shp", 
    'layers': [2018],           
    'method': "kmeans",                          # Aspatial Clustering: affinity_propagation, gaussian_mixture, hdbscan, kmeans, spectral, ward
                                                 # Spatial Clustering: azp, max_p, skater, spenc, ward_spatial   
    'nClusters': 6,                              # This option should be commented out for affinity_propagation, hdbscan and max_p 
    'variables': [     #Variables are column headers of your CSV
            "Median monthly housing costs",
            "% below poverty",
            "% unemployed",
            "% with 4year college degree",
            "% manufacturing",
            "% service industry",
            "% structures more than 30 years old",
            "% households moved <10 years ago",
            "% multiunit structures",
            "% owner occupied housing",
            "% vacant housing",
            "% > 60 years old",
            "% < 18 years old",
            "% white",
            "% Asian",
            "% Hispanic",
            "% black",
            "% foreign born"       
                 ],
    'Maps_of_Categorical_Data': True,     #Choropleth Maps visualizing categorical data
    'Stacked_Chart': False,               #Visualize Temporal Change over periods
    'Parallel_Categories_Diagram': False, #Quantifies Changes over multiple periods
    'Chord_Diagram': False                #Quantifies Change between two periods
}

### Run clustering using built-in data from geosnap (four periods)

In [96]:
param = {
    'title': "Longitudinal Neighborhood Change, Cook County (tract level)",
    'subject': "NEIGHBORHOOD",
    'filename_suffix': "Cook_Temporal_1980_1990_2000_2010",
    'state_fips': None,    #fips code is avaiable for every US state at https://github.com/suhanmappingideas/geosnap-viz/blob/master/state_id.csv
    'msa_fips': None,      #For more options: http://su-gis.iptime.org/LNE/pick_POI.html
    'county_fips': "17031",#county code is also availabe in the link right above.         
    'layers': [1980,1990,2000,2010],   # Available years: 1970, 1980, 1990, 2000 and 2010
    'method': "kmeans",               # Aspatial Clustering: affinity_propagation, gaussian_mixture, hdbscan, kmeans, spectral, ward
                                      # Spatial Clustering: azp, max_p, skater, spenc, ward_spatial   
    'nClusters': 6,                   # This option should be commented out for affinity_propagation, hdbscan and max_p 
    'variables': [  #Pick variables in the variable column from template/conversion_table_codebook.csv
            "p_nonhisp_white_persons",
            "p_nonhisp_black_persons",
            "p_hispanic_persons",
            "p_asian_persons",
            "p_foreign_born_pop",
            "p_edu_college_greater",
            "p_unemployment_rate",
            "p_employed_manufacturing",
            "p_poverty_rate",
            "p_vacant_housing_units",
            "p_owner_occupied_units",
            "p_housing_units_multiunit_structures",
            "median_home_value",
            "p_structures_30_old",
            "p_household_recent_move",
            "p_persons_under_18",
            "p_persons_over_60",        
                 ],
    #'Sequence': {'seq_clusters': 5, 'dist_type': 'tran'},
    'Maps_of_Categorical_Data': True,    #Choropleth Maps visualizing categorical data
    'Stacked_Chart': True,               #Visualize Temporal Change over periods
    'Parallel_Categories_Diagram': True, #Quantifies Changes over multiple periods
    'Chord_Diagram': False               #Quantifies Change between two periods
}

### Run clustering using built-in data from geosnap (two periods)

In [112]:
param = {
    'title': "Longitudinal Neighborhood Change, Cook County (tract level)",
    'subject': "NEIGHBORHOOD",
    'filename_suffix': "Cook_Temporal_1980_2010",              # "Albertville"
    'state_fips': None,    #fips code is avaiable for every US state at https://github.com/suhanmappingideas/geosnap-viz/blob/master/state_id.csv
    'msa_fips': None,      #For more options: http://su-gis.iptime.org/LNE/pick_POI.html
    'county_fips': "17031",#county code is also availabe in the link right above.
    'layers': [1980, 2010],          # Available years: 1970, 1980, 1990, 2000 and 2010
    'method': "kmeans",             # Aspatial Clustering: affinity_propagation, gaussian_mixture, hdbscan, kmeans, spectral, ward
                                    # Spatial Clustering: azp, max_p, skater, spenc, ward_spatial   
    'nClusters': 6,                 # This option should be commented out for affinity_propagation, hdbscan and max_p 
    'variables': [    #Pick variables in the variable column from template/conversion_table_codebook.csv
            "p_nonhisp_white_persons",
            "p_nonhisp_black_persons",
            "p_hispanic_persons",
            "p_asian_persons",
            "p_foreign_born_pop",
            "p_edu_college_greater",
            "p_unemployment_rate",
            "p_employed_manufacturing",
            "p_poverty_rate",
            "p_vacant_housing_units",
            "p_owner_occupied_units",
            "p_housing_units_multiunit_structures",
            "median_home_value",
            "p_structures_30_old",
            "p_household_recent_move",
            "p_persons_under_18",
            "p_persons_over_60",        
                 ],
    #'Sequence': {'seq_clusters': 5, 'dist_type': 'tran'},
    'Maps_of_Categorical_Data': True,    #Choropleth Maps visualizing categorical data
    'Stacked_Chart': True,               #Visualize Temporal Change over periods
    'Parallel_Categories_Diagram': True, #Quantifies Changes over multiple periods
    'Chord_Diagram': False               #Quantifies Change between two periods
}

### Run clustering using built-in data from geosnap (Sequence Analysis is included)

In [99]:
param = {
    'title': "Longitudinal Neighborhood Change, Cook County (tract level)",
    'subject': "NEIGHBORHOOD",
    'filename_suffix': "Cook_Temporal_1980_1990_2000_2010_Sequence",              
    'state_fips': None,    #fips code is avaiable for every US state at https://github.com/suhanmappingideas/geosnap-viz/blob/master/state_id.csv
    'msa_fips': None,      #For more options: http://su-gis.iptime.org/LNE/pick_POI.html
    'county_fips': "17031",#county code is also availabe in the link right above.     
    'layers': [1980,1990,2000,2010], # Available years: 1970, 1980, 1990, 2000 and 2010
    'method': "kmeans",             # Aspatial Clustering: affinity_propagation, gaussian_mixture, hdbscan, kmeans, spectral, ward
                                    # Spatial Clustering: azp, max_p, skater, spenc, ward_spatial   
    'nClusters': 6,                 # This option should be commented out for affinity_propagation, hdbscan and max_p 
    'variables': [                  #Pick variables in the variable column from template/conversion_table_codebook.csv    
            "p_nonhisp_white_persons",
            "p_nonhisp_black_persons",
            "p_hispanic_persons",
            "p_asian_persons",
            "p_foreign_born_pop",
            "p_edu_college_greater",
            "p_unemployment_rate",
            "p_employed_manufacturing",
            "p_poverty_rate",
            "p_vacant_housing_units",
            "p_owner_occupied_units",
            "p_housing_units_multiunit_structures",
            "median_home_value",
            "p_structures_30_old",
            "p_household_recent_move",
            "p_persons_under_18",
            "p_persons_over_60",        
                 ],
    'Sequence': {'seq_clusters': 5, 'dist_type': 'tran'},
    'Maps_of_Categorical_Data': True,    #Choropleth Maps visualizing categorical data
    'Stacked_Chart': True,               #Visualize Temporal Change over periods
    'Parallel_Categories_Diagram': True, #Quantifies Changes over multiple periods
    'Chord_Diagram': False               #Quantifies Change between two periods
}

In [126]:
# select community by state_fips, msa_fips, county_fips
community = None
if ('msa_fips' in param and param['msa_fips']):
    community = Community.from_ltdb(years=param['layers'], msa_fips=param['msa_fips'])
elif ('county_fips' in param and param['county_fips']):
    community = Community.from_ltdb(years=param['layers'], county_fips=param['county_fips'])
elif ('state_fips' in param and param['state_fips']):
    community = Community.from_ltdb(years=param['layers'], state_fips=param['state_fips'])

# if the user enters CSV and shapefile, use the files from the user

# This is executed when the user enter attributes in csv file and geometroy in shapefile ######################  
if (community is None and 'inputCSV' in param and param['inputCSV']):
    community = Community()
    community.gdf = pd.read_csv(param['inputCSV'], dtype={'geoid':str})
    geoid = community.gdf.columns[0]
    #community.gdf = community.gdf.astype(str)
    #print("inputCSV:  " + community.gdf.geoid)        
    community.gdf['geoid'] = community.gdf['geoid'].astype(str)
    #print("community.gdf.columns[0]:", community.gdf.columns[0])
    
    # read shape file to df_shape
    df_shape = gpd.read_file(param['shapefile'])
    df_shape = df_shape.astype(str)     
    #print("shapefile:  " + df_shape.GEOID10)        
    df_shape = df_shape.set_index("GEOID10")
    
    # insert geometry to community.gdf
    geometry = []
    for index, row in community.gdf.iterrows():
        tractid = row[geoid]
        try:
            tract = df_shape.loc[tractid]
            geometry.append(shapely.wkt.loads(tract.geometry))
        except KeyError:
            #print("Tract ID [{}] is not found in the shape file {}".format(tractid, param['shapefile']))
            geometry.append(None)
    community.gdf.insert(len(community.gdf.columns), "geometry", geometry)
    # need to change layer to year for clustering computation in the later stage
    community.gdf.rename(columns={'layer':'year'}, inplace=True)
################################################################################################################      
    
community.gdf = community.gdf.replace([np.inf, -np.inf], np.nan)

# check if geometry is not null for Spatial Clustering
community.gdf = community.gdf[pd.notnull(community.gdf['geometry'])]

#community.gdf.to_csv('LTDB_2018_1990_2000_2010__tract_Cook_byTract_normalized.csv')
community.gdf

,geoid,year,Median monthly housing costs,% with 4year college degree,% unemployed,% manufacturing,% below poverty,% structures more than 30 years old,% households moved <10 years ago,% > 60 years old,...,"% housing units 1-unit, attached",% housing units 2 units,% housing units 3 or 4 units,% housing units 5 to 9 units,% housing units 10 to 19 units,% housing units 20 or more units,% housing units mobile home,"% housing units boat, rv, van, etc",% housing units two or more,geometry
0,60004,2018,1531.0,36.776332,2.355577,11.776084,4.782419,63.965129,38.627019,26.247668,...,8.526129,0.440676,3.793649,11.855152,3.151794,12.741294,0.000000,0.000000,20.127413,POLYGON ((-87.98511299996494 42.14212399962292...
1,60005,2018,1328.0,32.387085,3.490593,14.569493,7.904073,76.050294,44.809547,27.401063,...,4.457160,1.084174,1.927421,9.140190,5.435928,29.927722,1.769312,0.165638,38.375245,(POLYGON ((-87.9542779998298 42.02977200037759...
2,60007,2018,1358.0,23.466325,3.615668,13.549801,3.828198,66.471051,33.641789,26.196888,...,9.432477,0.407930,3.485293,8.494955,3.692836,15.501324,0.128820,0.000000,23.087383,POLYGON ((-88.01774700032377 41.99246799959855...
3,60008,2018,1372.0,22.906752,3.823682,10.223113,6.285767,76.331768,46.627566,20.189621,...,8.603506,0.607171,4.009623,5.911330,7.114217,25.168977,0.000000,0.000000,36.899989,(POLYGON ((-88.06229599976052 42.0772629996877...
4,60010,2018,2431.0,35.542280,2.296360,12.766159,4.214367,48.356571,33.793164,27.452427,...,7.413629,1.528641,2.228568,1.383056,0.425556,4.865894,0.145585,0.000000,9.048659,POLYGON ((-88.05268299982055 42.15381099987781...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,60714,2018,1166.0,22.419524,4.338918,9.268662,9.818641,81.559742,39.263006,32.730390,...,8.774610,1.612356,5.465037,7.764766,1.009844,21.469790,0.432790,0.000000,29.557026,(POLYGON ((-87.84536600004083 42.0544489998215...
185,60803,2018,1065.0,13.532678,6.099911,10.749155,10.313018,68.412787,38.123351,19.776686,...,4.778463,0.897364,5.574874,14.357824,12.462142,7.392036,2.882782,0.000000,26.326416,POLYGON ((-87.74340699983316 41.69078599997397...
186,60804,2018,983.0,7.004462,4.599960,20.843393,18.172980,95.407209,48.538558,11.636018,...,2.391994,31.462045,17.044992,6.748841,3.059149,2.237409,0.451550,0.000000,53.803596,POLYGON ((-87.77800199979242 41.82887399983525...
187,60805,2018,1383.0,21.959485,2.818781,7.529461,7.201974,92.258153,35.186611,19.348114,...,1.383941,2.562346,2.014250,7.111537,1.534667,2.096465,0.000000,0.000000,8.207728,"POLYGON ((-87.70254499955206 41.7061740000301,..."


## Write index.html

In [127]:
#Create a new folder where GEO_CONFIG.js GEO_JSON.js VARIABLES.js will be saved
oDir = 'QUAL_' + param['filename_suffix']
path = Path(oDir + '/data')
path.mkdir(parents=True, exist_ok=True)

contents = []
#open Neighborhood_Analysis_Mapper.html (the excutable file for the visualization)
ifile = open("template/Qualitative_Analysis_Mapper.html", "r", encoding="utf-8")
contents = ifile.read()

#Replace variables based on the user's selection in each of four files below.
contents = contents.replace("Neighborhood Analysis Mapper", param['title'])
contents = contents.replace("data/CONFIG.js", "data/CONFIG_"+param['filename_suffix']+".js")
contents = contents.replace("data/GEO_JSON.js", "data/GEO_JSON_"+param['filename_suffix']+".js")
contents = contents.replace("data/VARIABLES.js", "data/VARIABLES_"+param['filename_suffix']+".js")

#write new outfiles: GEO_CONFIG.js GEO_JSON.js VARIABLES.js
ofile = open(oDir+"/index.html", "w", encoding="utf-8")
ofile.write(contents)
ofile.close()

cwd = os.getcwd()
fname =urllib.parse.quote('index.html')
template_dir = os.path.join(cwd, 'QUAL_' + param['filename_suffix'])
url = os.path.join(template_dir, fname)    
print("Your HTML file has been created. Here is your html file: " + url)

Your HTML file has been created. Here is your html file: F:\Github\MapLinksPlot\PYTHON_Categorical_Data_VIZ\QUAL_Cook_2018_from_ACS_zipcode\index.html


## Write GEO_CONFIG_XXX.js

In [128]:
# read ACM_GEO_CONFIG.js
ifile = open("template/QUAL_CONFIG.js", "r", encoding="utf-8")
contents = ifile.read()

SubjectName = "";
Maps_of_neighborhood = True;               
Temporal_change_in_neighborhoods = True;
Parallel_Categories_Diagram_in_neighborhoods = True;
Chord_Diagram_in_neighborhoods = True;

if ('subject' in param): SubjectName =  param['subject']
if ('Maps_of_Categorical_Data' in param): Maps_of_Categorical_Data =  param['Maps_of_Categorical_Data']
if ('Stacked_Chart' in param): Stacked_Chart =  param['Stacked_Chart']
if ('Parallel_Categories_Diagram' in param): Parallel_Categories_Diagram =  param['Parallel_Categories_Diagram']
if ('Chord_Diagram' in param): Chord_Diagram =  param['Chord_Diagram']

# perpare parameters
NumOfMaps = len(param['layers'])

InitialLayers = []
if (len(param['layers']) <= 1): InitialLayers = []
for i, layer in enumerate(param['layers']):
	InitialLayers.append(str(layer))
InitialLayers

['2018']

In [129]:
NumOfMaps = len(InitialLayers)
# Automatically set Map_width, Map_height. 
Map_width = "300px"
Map_height = "300px"
if (NumOfMaps <= 6):
    Map_width = "300px"
    Map_height = "300px"    
if (NumOfMaps <= 5):
    Map_width = "350px"
    Map_height = "350px"
if (NumOfMaps <= 4):
    Map_width = "400px"
    Map_height = "400px"
if (NumOfMaps <= 3):
    Map_width = "400px"
    Map_height = "400px"
if (NumOfMaps <= 2):
    Map_width = "450px"
    Map_height = "450px"
if (NumOfMaps ==    1):
    Map_width = "800px"
    Map_height = "800px"
    
# replace newly computed "NumOfMaps", "InitialLayers", "Map_width", "Map_height" in CONFIG.js. See the example replacement below
InitialLayers = "var InitialLayers = " + json.dumps(InitialLayers) + ";"
SubjectName = 'var SubjectName = "' + SubjectName + '";'
Maps_of_Categorical_Data = "var Maps_of_Categorical_Data = " + json.dumps(Maps_of_Categorical_Data)+ ";"
Stacked_Chart = "var Stacked_Chart = " + json.dumps(Stacked_Chart)+ ";"
Parallel_Categories_Diagram = "var Parallel_Categories_Diagram = " + json.dumps(Parallel_Categories_Diagram)+ ";"
Chord_Diagram = "var Chord_Diagram = " + json.dumps(Chord_Diagram)+ ";"
Map_width = 'var Map_width  = "' + Map_width + '";'
Map_height = 'var Map_height = "' + Map_height + '";'

contents = contents.replace("var InitialLayers = [];", InitialLayers)
contents = contents.replace('var SubjectName = "";', SubjectName)
contents = contents.replace("var Maps_of_Categorical_Data = true;", Maps_of_Categorical_Data)
contents = contents.replace("var Stacked_Chart = true;", Stacked_Chart)
contents = contents.replace("var Parallel_Categories_Diagram = true;", Parallel_Categories_Diagram)
contents = contents.replace("var Chord_Diagram = true;", Chord_Diagram)
contents = contents.replace('var Map_width  = "400px";', Map_width)
contents = contents.replace('var Map_height = "400px";', Map_height)

#Write output including the replacement above
filename_GEO_CONFIG = "QUAL_" + param['filename_suffix'] + "/data/CONFIG_"+param['filename_suffix']+".js"
ofile = open(filename_GEO_CONFIG, 'w', encoding="utf-8")
ofile.write(contents)
ofile.close()    
print (contents) 

// Define the number of maps and some configuration parameters that you want to visualize.
var SubjectName = "NEIGHBORHOOD";
var InitialLayers = ["2018"];

/* Map Extent and Zoom level will be automatically adjusted when you do not define map center and zoom level */
//var Initial_map_center = [34.0522, -117.9];  
//var Initial_map_zoom_level = 8;   

var Maps_of_Categorical_Data = true;							//choropleth map: Maps representing categorical data 
var Stacked_Chart = false;				//stacked chart: Temporal Change
var Parallel_Categories_Diagram = false;	//parallel categories diagram
var Chord_Diagram = false;					//chord diagram
  
var Num_Of_Decimal_Places = 2;                             // default = 2

var Map_width  = "800px";                                  // min 350px
var Map_height = "800px";                                  // min 300px


## Write GEO_JSON_XXX.js

In [130]:
# query geometry for each tract
geoid = community.gdf.columns[0]
tracts = community.gdf[[geoid, 'geometry']].copy()
tracts.drop_duplicates(subset=geoid, inplace=True)					# get unique geoid
tracts

,geoid,geometry
0,60004,POLYGON ((-87.98511299996494 42.14212399962292...
1,60005,(POLYGON ((-87.9542779998298 42.02977200037759...
2,60007,POLYGON ((-88.01774700032377 41.99246799959855...
3,60008,(POLYGON ((-88.06229599976052 42.0772629996877...
4,60010,POLYGON ((-88.05268299982055 42.15381099987781...
...,...,...
184,60714,(POLYGON ((-87.84536600004083 42.0544489998215...
185,60803,POLYGON ((-87.74340699983316 41.69078599997397...
186,60804,POLYGON ((-87.77800199979242 41.82887399983525...
187,60805,"POLYGON ((-87.70254499955206 41.7061740000301,..."


In [131]:
# open GEO_JSON.js write heading for geojson format
filename_GEO_JSON = "QUAL_" + param['filename_suffix'] + "/data/GEO_JSON_"+param['filename_suffix']+".js"
ofile = open(filename_GEO_JSON, 'w')
ofile.write('var GEO_JSON =\n')
ofile.write('{"type":"FeatureCollection", "features": [\n')

#Convert geometry in GEOJSONP to geojson format
wCount = 0
for tract in tracts.itertuples():
	feature = {"type":"Feature"}
	if (type(tract.geometry) is float):								# check is NaN?
		#print(tract.geometry)
		continue
	#print(tract.geometry)        
	feature["geometry"] = shapely.geometry.mapping(tract.geometry)
	#feature["properties"] = {geoid: tract.__getattribute__(geoid), "tractID": tract.__getattribute__(geoid)}
	feature["properties"] = {geoid: tract.__getattribute__(geoid)}
	wCount += 1
	ofile.write(json.dumps(feature)+',\n')
#print("GEO_JSON.js write count:", wCount)
# complete the geojosn format by adding parenthesis at the end.	
ofile.write(']}\n')
ofile.close()

cwd = os.getcwd()
print("Your geometry file has been created. Here is your JS file: " + cwd + '/'+ 'QUAL_' + param['filename_suffix']+'/data/GEO_CONFIG_' + param['filename_suffix']+'.js')

Your geometry file has been created. Here is your JS file: F:\Github\MapLinksPlot\PYTHON_Categorical_Data_VIZ/QUAL_Cook_2018_from_ACS_zipcode/data/GEO_CONFIG_Cook_2018_from_ACS_zipcode.js


## Write GEO_VARIABLES_XXXX.js

### Clustering Algorithm runs in the cell below. It might take a while depending on what clustring algorithm you picked

In [132]:
geoid       = community.gdf.columns[0]
method      = param['method']
nClusters   = param['nClusters']
years       = param['layers']
variables   = param['variables']

# filtering by years
community.gdf = community.gdf[community.gdf.year.isin(years)]
#community.gdf.to_csv(r'output.csv')   

if (method == 'kmeans' or method == 'ward' or method == 'affinity_propagation' or method == 'spectral' or method == 'gaussian_mixture' or method == 'hdbscan'):
	clusters = community.cluster(columns=variables, method=method, n_clusters=nClusters)
if (method == 'ward_spatial' or method == 'spenc' or method == 'skater' or method == 'azp' or method == 'max_p'):
	clusters = community.cluster_spatial(columns=variables, method=method, n_clusters=nClusters)		

# Use the sequence method to obtain the distance matrix of neighborhood sequences
gdf_new, df_wide, seq_dis_mat = clusters.sequence(dist_type='tran', cluster_col=method)
df_wide

c:\users\suzie\.conda\envs\geosnap\lib\site-packages\sklearn\cluster\_kmeans.py:934: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 0.25. It has no effect
  "effect", FutureWarning)
c:\users\suzie\.conda\envs\geosnap\lib\site-packages\sklearn\cluster\_kmeans.py:939: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)
c:\users\suzie\.conda\envs\geosnap\lib\site-packages\scipy\cluster\hierarchy.py:826: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


year,2018,tran_5
geoid,,
60004,1,1
60005,5,2
60007,1,1
60008,5,2
60010,1,1
...,...,...
60714,5,2
60803,5,2
60804,0,0


In [133]:

if (method == 'kmeans' or method == 'ward' or method == 'affinity_propagation' or method == 'spectral' or method == 'gaussian_mixture' or method == 'hdbscan'):
	clusters = community.cluster(columns=variables, method=method, n_clusters=nClusters)
if (method == 'ward_spatial' or method == 'spenc' or method == 'skater' or method == 'azp' or method == 'max_p'):
	clusters = community.cluster_spatial(columns=variables, method=method, n_clusters=nClusters)		

# Use the sequence method to obtain the distance matrix of neighborhood sequences
gdf_new, df_wide, seq_dis_mat = clusters.sequence(dist_type='tran', cluster_col=method)
df_wide

c:\users\suzie\.conda\envs\geosnap\lib\site-packages\sklearn\cluster\_kmeans.py:934: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 0.25. It has no effect
  "effect", FutureWarning)
c:\users\suzie\.conda\envs\geosnap\lib\site-packages\sklearn\cluster\_kmeans.py:939: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)
c:\users\suzie\.conda\envs\geosnap\lib\site-packages\scipy\cluster\hierarchy.py:826: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


year,2018,tran_5
geoid,,
60004,5,1
60005,0,2
60007,5,1
60008,0,2
60010,5,1
...,...,...
60714,0,2
60803,0,2
60804,2,0


In [134]:
df_pivot = df_wide
lastColumn = df_pivot.columns[df_pivot.shape[1]-1]					# get the last column name as like 'tran-5'
df_pivot.rename(columns={lastColumn: 'Sequence'}, inplace=True)		# change the last column name to 'Sequence'
df_pivot
#df_pivot.to_csv(r'df_pivot.csv')  

year,2018,Sequence
geoid,,
60004,5,1
60005,0,2
60007,5,1
60008,0,2
60010,5,1
...,...,...
60714,0,2
60803,0,2
60804,2,0


In [135]:
if ('Sequence' not in param or not param['Sequence']): df_pivot.drop(columns=['Sequence'], inplace=True)
df_pivot
#df_pivot.to_csv(r'df_pivot.csv') 

year,2018
geoid,
60004,5
60005,0
60007,5
60008,0
60010,5
...,...
60714,0
60803,0
60804,2


In [136]:
# write df_wide to GEO_VARIABLES.js
filename_GEO_VARIABLES = "QUAL_" + param['filename_suffix'] + "/data/VARIABLES_"+param['filename_suffix']+".js"
ofile = open(filename_GEO_VARIABLES, 'w')
ofile.write('var GEO_VARIABLES =\n')
ofile.write('[\n')

heading = [geoid]
heading.extend(list(map(str, df_pivot.columns.tolist())))
ofile.write('  '+json.dumps(heading)+',\n')
wCount = 0
for i, row in df_pivot.reset_index().iterrows():
	aLine = row.tolist()
	for j, col in enumerate(aLine[2:], 2):
		try:
			aLine[j] = int(col)                                  # convert float to int
		except ValueError:
			aLine[j] = -9999                                     # if Nan, set -9999
	wCount += 1 
	ofile.write('  '+json.dumps(aLine)+',\n')
#print("GEO_VARIABLES.js write count:", wCount)
ofile.write(']\n')
ofile.close()

#Print what's inside GEO_VARIABLES.js
with open(filename_GEO_VARIABLES, 'r') as f2:
    data = f2.read()
    print(data)

var GEO_VARIABLES =
[
  ["geoid", "2018"],
  ["60004", 5],
  ["60005", 0],
  ["60007", 5],
  ["60008", 0],
  ["60010", 5],
  ["60015", 5],
  ["60016", 0],
  ["60018", 0],
  ["60022", 5],
  ["60025", 5],
  ["60026", 5],
  ["60029", 5],
  ["60035", 5],
  ["60043", 5],
  ["60047", 5],
  ["60053", 5],
  ["60056", 0],
  ["60062", 5],
  ["60067", 5],
  ["60068", 5],
  ["60070", 0],
  ["60074", 0],
  ["60076", 0],
  ["60077", 0],
  ["60089", 5],
  ["60090", 0],
  ["60091", 5],
  ["60093", 5],
  ["60103", 5],
  ["60104", 3],
  ["60106", 0],
  ["60107", 0],
  ["60110", 2],
  ["60118", 5],
  ["60120", 2],
  ["60126", 5],
  ["60130", 0],
  ["60131", 0],
  ["60133", 0],
  ["60143", 5],
  ["60153", 3],
  ["60154", 5],
  ["60155", 3],
  ["60157", 5],
  ["60160", 2],
  ["60162", 3],
  ["60163", 0],
  ["60164", 2],
  ["60165", 2],
  ["60169", 0],
  ["60171", 0],
  ["60172", 5],
  ["60173", 4],
  ["60176", 0],
  ["60191", 0],
  ["60192", 5],
  ["60193", 5],
  ["60194", 0],
  ["60195", 4],
  ["60201", 4

### Creating URL to view the visualizatino result

In [137]:
servers = list(notebookapp.list_running_servers())
servers1 = 'https://cybergisx.cigi.illinois.edu'+servers[0]["base_url"]+ 'view'
servers2 = 'https://cybergisx.cigi.illinois.edu'+servers[0]["base_url"]+ 'edit'      
cwd = os.getcwd()
prefix_cwd = "/home/jovyan/work"
cwd = cwd.replace(prefix_cwd, "")

# This is for Jupyter notebbok installed in your PC
local_dir1 = cwd
local_dir2 = cwd  

#This is for CyberGISX. Uncomment two command lines below when you run in CyberGIX Environment
#local_dir1 = servers1 + cwd
#local_dir2 = servers2 + cwd   

#print(local_dir)
fname =urllib.parse.quote('index.html')
template_dir = os.path.join(local_dir1, 'QUAL_' + param['filename_suffix'])
#url = 'file:' + os.path.join(template_dir, fname)
url = os.path.join(template_dir, fname)    
webbrowser.open(url)
print('To see visualization of your analysis, click the URL below:')
print(url)    
print('Advanced options are available in ')  
print(local_dir2 + '/'+ 'QUAL_' + param['filename_suffix']+'/data/GEO_CONFIG_' + param['filename_suffix']+'.js')

To see visualization of your analysis, click the URL below:
F:\Github\MapLinksPlot\PYTHON_Categorical_Data_VIZ\QUAL_Cook_2018_from_ACS_zipcode\index.html
Advanced options are available in 
F:\Github\MapLinksPlot\PYTHON_Categorical_Data_VIZ/QUAL_Cook_2018_from_ACS_zipcode/data/GEO_CONFIG_Cook_2018_from_ACS_zipcode.js
